In [ ]:
import io, json, os, re, requests, socket, subprocess

import pandas as pd

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
notebook_wide_display()

In [ ]:
# Read hosts to ping from Google Sheets
googleSheetsUrl = 'https://docs.google.com/spreadsheets/d/1VSVIODqpBcFn6HtuawzAQGtUCewrzPK33_lcXoXesYk/edit#gid=0'

In [ ]:
def sheetUrl2CsvUrl(sheetUrl):
    tokens = sheetUrl.split('/')
    assert(len(tokens) == 7)
    assert(tokens[4] == 'd')
    docHash = tokens[5]
    assert(len(docHash) > 20)
    edit = tokens[6]
    assert edit[0:9] == 'edit#gid='
    gid = edit[9:]
    return 'https://docs.google.com/spreadsheets/d/' + docHash + '/export?format=csv&gid=' + gid

def countPings(dest, count):
    cmd = 'ping -n -c %d %s' % (count, dest)
    response = subprocess_check(cmd, ignore_error=True)
    match = re.search(r'\d+ packets transmitted.*?(\d+) received', response)
    if match:
        return int(match.group(1))
    else:
        raise Exception('countPings could not parse response: %s' % response)

def monitorOnce():  
    sheetContent = requests.get(sheetUrl2CsvUrl(googleSheetsUrl)).content
    configMsg = '<a href="%s">Config</a>' % googleSheetsUrl
    df = pd.read_csv(io.BytesIO(sheetContent), na_filter=False)
    hostname = socket.gethostname()

    # Only use the records with our hostname as Tester
    df = df[df.Tester == hostname]
    
    nSendPings = 5
    nMinReceivePings = 4
    nErrors = 0
    for index, row in df.iterrows():
        try:
            target = row['Target'].strip()
            displayname = row['Nickname'].strip() or row['Target'].strip()
            service = row['Service'].strip()
            if target.startswith('http://') or target.startswith('https://'):
                try:
                    response = requests.get(target, timeout=10)
                    response.raise_for_status()
                    Stat.up('%s is up' % displayname,
                            '%s answered %d bytes (status %d), requested from %s. %s' % 
                            (target, len(response.content), response.status_code, hostname, configMsg),
                            host=displayname,
                            service=service)
                except Exception as e:
                    Stat.down('%s is down' % displayname,
                              'Exception %s while attemping GET %s, requested from %s. %s' % 
                              (e, target, hostname, configMsg),
                              host=displayname,
                              service=service)
            else:
                nReceived = countPings(target, nSendPings)
                if nReceived >= nMinReceivePings:
                    Stat.up('%s is up' % displayname, 
                            '%s answered %d of %d pings from %s. %s' % 
                            (target, nReceived, nSendPings, hostname, configMsg),
                            host=displayname,
                            service=service)
                else:
                    Stat.down('%s is down' % displayname, 
                              '%s answered %d of %d pings from %s. %s' %
                              (target, nReceived, nSendPings, hostname, configMsg),
                              host=displayname,
                              service=service)
                              
        except Exception as e:
            Stat.critical('Exception %s while monitoring %s' % (e, displayname), service='statping')
            nErrors += 1
    if nErrors:
        Stat.down('statping caught %d exceptions in monitorOnce' % nErrors, 
                  configMsg,
                  service='statping')
    else:
        Stat.up('statping is monitoring %d sites' % len(df), 
                configMsg,
                service='statping')

In [ ]:
while True:
    monitorOnce()
    sleep_until_next_period(60)